# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
path = '../WeatherPy/output_data/cities.csv'
df = pd.read_csv(path)
df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Northam,51.0333,-4.2167,61.61,94,100,8.75,GB,1666215251
1,1,Port Macquarie,-31.4333,152.9167,64.89,100,100,5.75,AU,1666215093
2,2,Ushuaia,-54.8000,-68.3000,40.66,87,75,24.16,AR,1666215132
3,3,Kapaa,22.0752,-159.3190,76.98,86,100,14.97,US,1666215074
4,4,Cape Town,-33.9258,18.4232,64.81,72,75,10.36,ZA,1666214987
...,...,...,...,...,...,...,...,...,...,...
575,575,Samarai,-10.6167,150.6667,83.62,75,22,12.26,PG,1666215188
576,576,Shache,38.4167,77.2406,53.71,38,100,10.36,CN,1666215391
577,577,Sabha,27.0377,14.4283,70.90,44,82,4.14,LY,1666215391
578,578,Kabardinka,44.6514,37.9428,59.25,81,1,5.82,RU,1666215392


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
# Set locations and weight
locations = df[['Lat', 'Lng']].astype(float)
weight = df['Humidity'].astype(float)

In [9]:
# Create humidity heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
perfect_df = df.loc[(df['Max Temp (F)'] <= 80) & (df['Max Temp (F)'] >= 70)\
                    & (df['Wind Speed'] < 10) & (df['Cloudiness'] == 0)]
perfect_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
172,172,San Patricio,28.0170,-97.5169,74.17,23,0,6.40,US,1666215191
208,208,Karratha,-20.7377,116.8463,73.80,72,0,7.67,AU,1666215192
214,214,Farmington,36.7281,-108.2187,73.20,29,0,8.05,US,1666215295
390,390,Salalah,17.0151,54.0924,77.09,65,0,1.14,OM,1666215170
440,440,Yakima,46.4665,-120.6678,76.21,36,0,3.44,US,1666214910


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# Create hotel_df
hotel_df = perfect_df.reindex(columns = perfect_df.columns.tolist() + ['Hotel Name'])
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
172,172,San Patricio,28.0170,-97.5169,74.17,23,0,6.40,US,1666215191,NaN
208,208,Karratha,-20.7377,116.8463,73.80,72,0,7.67,AU,1666215192,NaN
214,214,Farmington,36.7281,-108.2187,73.20,29,0,8.05,US,1666215295,NaN
390,390,Salalah,17.0151,54.0924,77.09,65,0,1.14,OM,1666215170,NaN
440,440,Yakima,46.4665,-120.6678,76.21,36,0,3.44,US,1666214910,NaN


In [17]:
# Hit the Google Places API for each city's coordinates and store them in empty list
lat_data = []
lng_data = []

for index, row in hotel_df.iterrows():
    city = row['City']
    country = row['Country']
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city},{country}&key={g_key}"
    
    response = requests.get(target_url).json()
    
    
    lat_data.append(response["results"][0]["geometry"]["location"]["lat"])
    lng_data.append(response["results"][0]["geometry"]["location"]["lng"]) 

In [18]:
# Update DataFrame with Google's coordinates of each city 
lat_col = hotel_df.columns[2]
lng_col = hotel_df.columns[3]
hotel_df[lat_col] = lat_data
hotel_df[lng_col] = lng_data
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
172,172,San Patricio,27.999969,-97.524724,74.17,23,0,6.40,US,1666215191,NaN
208,208,Karratha,-20.733695,116.844706,73.80,72,0,7.67,AU,1666215192,NaN
214,214,Farmington,41.736030,-72.795027,73.20,29,0,8.05,US,1666215295,NaN
390,390,Salalah,17.019384,54.110750,77.09,65,0,1.14,OM,1666215170,NaN
440,440,Yakima,46.602071,-120.505899,76.21,36,0,3.44,US,1666214910,NaN


In [19]:
# Set parameters to search for nearby hotels

params = {
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [20]:
# Loop through DataFrame to get the nearest hotel and add it to the df
for index, row in hotel_df.iterrows():
    
#     Get location from df
    lat_cord = row['Lat']
    lng_cord = row['Lng']
    
#     Add location to params dict
    params['location'] = f"{lat_cord},{lng_cord}"
    
#     Make API request
    response = requests.get(base, params=params).json()
    
#     Add hotel name to the df
    hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
172,172,San Patricio,27.999969,-97.524724,74.17,23,0,6.40,US,1666215191,Americas Best Value Inn Sinton
208,208,Karratha,-20.733695,116.844706,73.80,72,0,7.67,AU,1666215192,ibis Styles Karratha
214,214,Farmington,41.736030,-72.795027,73.20,29,0,8.05,US,1666215295,The Farmington Inn & Suites
390,390,Salalah,17.019384,54.110750,77.09,65,0,1.14,OM,1666215170,Darbat Hotel
440,440,Yakima,46.602071,-120.505899,76.21,36,0,3.44,US,1666214910,The Hotel Y


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))